# Desarrollo de las funciones de la API.

In [1]:
import pandas as pd

In [16]:
# @app.get("/")
def peliculas_idioma(idioma:str):
    df = pd.read_csv('clean_data/movies.csv')
    df = df[df.original_language == idioma]
    return print(f'La cantidad de películas producidas originalmente en "{idioma}" son: {len(df)}')

# @app.get("/")
def peliculas_duración(pelicula:str):
    df = pd.read_csv('clean_data/movies.csv')
    row = df[df.title == pelicula]
    duracion = row.runtime.item()
    anio = row.release_year.item()
    return print(f'La pelicula {pelicula} tiene una duración de {duracion} minutos. Año {anio}')

# @app.get("/")
def franquicia(franquicia:str):
    df = pd.read_csv('clean_data/franquicias.csv')
    dfx = pd.read_csv('clean_data/franq_movies.csv')
    dfy = pd.read_csv('clean_data/movies.csv')
    df = df[df.franquicia == franquicia]
    df = df.merge(dfx)[['id_peli','franquicia']]
    df = df.merge(dfy)[['title','franquicia','revenue','budget']]
    df['ganancia'] = df.revenue - df.budget
    g_total = df.ganancia.sum()
    g_promedio = df.ganancia.mean()
    peliculas = len(df)
    return print(f'La franquicia {franquicia} tiene un total de {peliculas} peliculas, una ganancia total de {g_total} y una ganancia promedio de {g_promedio}') 

# @app.get("/")
def peliculas_pais(pais:str):
    df = pd.read_csv('clean_data/paises.csv')
    dfx = pd.read_csv('clean_data/pais_movies.csv')
    df = df[df.pais == pais]
    df = df.merge(dfx)[['id_peli']]
    peliculas = len(df)
    return print(f'Se producieron {peliculas} peliculas en el pais {pais}')

# @app.get("/")
def productoras_exitosas(productora:str):
    df = pd.read_csv('clean_data/productoras.csv')
    dfx = pd.read_csv('clean_data/prod_movies.csv')
    dfy = pd.read_csv('clean_data/movies.csv')
    df = df[df.productora == productora]
    df = df.merge(dfx)[['id_peli','productora']]
    df = df.merge(dfy)[['productora','revenue']]
    peliculas = len(df)
    rev_total = df.revenue.sum()
    return print(f'La productora {productora} tiene {peliculas} peliculas con un revenue total de {rev_total} entre todas')

# @app.get("/")
def get_director(director:str):
    df = pd.read_csv('clean_data/directores.csv')
    dfx = pd.read_csv('clean_data/dir_movies.csv')
    dfy = pd.read_csv('clean_data/movies.csv')
    df = df[df.director == director]
    df = df.merge(dfx)[['id_peli','director']]
    df = df.merge(dfy)[['title','release_date','revenue','budget','return']]
    retorno = df.revenue.sum()/df.budget.sum()
    listas = df.to_numpy().tolist()
    return print(f'El director {director} tiene un éxito medido a través del retorno de todas sus películas (esto es: df.revenue.sum()/df.budget.sum()) de {retorno}.\n\nLas películas que dirigió son:\n\n{listas}', "Se utilizó el siguiente calculo para determinar el retorno: df.revenue.sum()/df.budget.sum()")

In [17]:
peliculas_duración('The Doom Generation')

La pelicula The Doom Generation tiene una duración de 80.0 minutos. Año 1995
